In [1]:
!pip install opencv-python

In [2]:
import numpy as np
import cv2

In [3]:
def get_background(file_path):
    cap = cv2.VideoCapture(file_path)
    # we will randomly select 50 frames for the calculating the median
    frame_indices = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=50)
    # we will store the frames in array
    frames = []
    for idx in frame_indices:
        # set the frame id to read that particular frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        frames.append(frame)
    # calculate the median
    median_frame = np.median(frames, axis=0).astype(np.uint8)
    return median_frame

In [4]:
INPUT_VIDEO_PATH = "input_files/Traffic_Laramie_1.mp4"
CONSECUTIVE_FRAMES = 2 # how manny frames will be compared to detect movement, less = more detailed detection, more computation needed
MINIMAL_SIZE = 2000 # minimal object size, big enough to detect cars but avoid pedestrians

In [ ]:
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)
# get the video frame height and width
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(5))

save_name = f"output_files_task2/{INPUT_VIDEO_PATH.split('/')[-1]}"
# define codec and create VideoWriter object
out = cv2.VideoWriter(
    save_name,
    cv2.VideoWriter_fourcc(*'mp4v'), fps, 
    (frame_width, frame_height)
)